In [1]:
# In a Jupyter Notebook cell
%load_ext autoreload
%autoreload 2

from mirage.index import WordCountingChunkingAlgorithm, FolderRawStorage, SQLiteChunkStorage, RAMChunkStorage
from mirage.embedders.TextNormallizer import TextNormallizer
import os
import pickle

# Your code here

In [9]:
# if os.path.exists('temp_ram_storage'):
#     with open('temp_ram_storage', 'rb') as f:
#         chunks = pickle.load(f)
# with open('temp_ram_storage', 'wb') as f:
#     pickle.dump(chunks, f)
# else:
# 

import logging
logging.basicConfig(level=logging.DEBUG)


documents = FolderRawStorage('data')
chunks = RAMChunkStorage()
algorithm = WordCountingChunkingAlgorithm(documents, chunks, words_amount=10)
algorithm.execute()



INFO:mirage.index.chunking_algorithms.ChunkingAlgorithm:Chunking of documents


['1.txt.txt', '2.txt.txt', '3.txt.txt', '4.txt.txt', '5.txt.txt']
set()
entering a main loop


  0%|          | 0/5 [00:00<?, ?it/s]INFO:mirage.index.chunking_algorithms.WordCountingChunkAlgorithm: 90 words are parsed from the document
INFO:mirage.index.chunking_algorithms.WordCountingChunkAlgorithm: 89 words are parsed from the document
INFO:mirage.index.chunking_algorithms.WordCountingChunkAlgorithm: 90 words are parsed from the document
INFO:mirage.index.chunking_algorithms.WordCountingChunkAlgorithm: 87 words are parsed from the document
INFO:mirage.index.chunking_algorithms.WordCountingChunkAlgorithm: 91 words are parsed from the document
100%|██████████| 5/5 [00:00<00:00, 1666.39it/s]

7564616747055211180 Самолет летит высоко в небе. Самолеты часто перевозят пассажиров. Самолетный
-7729341251079173021 шум слышен издалека. Автомобиль едет по дороге. Автомобили бывают разных
-1037449722206109400 цветов. Автомобильный двигатель работает громко. Поезд движется по рельсам. Поезда
-5967334848183472705 перевозят грузы и людей. Поездный вокзал всегда busy. Велосипед стоит
3801711171415958650 у дома. Велосипеды популярны в городе. Велосипедная прогулка полезна для
-5411010187642328620 здоровья. Корабль плывет по морю. Корабли бывают большими и маленькими.
3545096870229102315 Корабельная команда работает слаженно. Трамвай едет по городу. Трамваи удобны
2201076803617522514 для поездок. Трамвайный маршрут проходит через центр. Автобус остановился на
-4075323192343073511 Дерево растет в лесу. Деревья дают тень в жаркий день.
-3853568377787044718 Деревянный дом выглядит уютно. Река течет быстро. Реки впадают в
-3343117601623064829 моря. Речной берег покрыт песком. Гора возвышается

5

In [79]:
import random
indexes = chunks.get_indexes()
normalizer = TextNormallizer(stop_word_remove=True, word_generalization='stem')
index = random.choice(list(indexes))
print(index)
print(normalizer.normalize(chunks[index]))
print(chunks[index])


INFO:pymorphy2.opencorpora_dict.wrapper:Loading dictionaries from C:\Users\sk2va\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymorphy2_dicts_ru\data
INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


-5967334848183472705
перевоз груз люд поездн вокза busi велосипед сто
перевозят грузы и людей. Поездный вокзал всегда busy. Велосипед стоит


In [80]:
from mirage.embedders.BowEmbedder import BowEmbedder

emb = BowEmbedder(normalizer = True)
emb.fit(chunks)

INFO:pymorphy2.opencorpora_dict.wrapper:Loading dictionaries from C:\Users\sk2va\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\pymorphy2_dicts_ru\data
INFO:pymorphy2.opencorpora_dict.wrapper:format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [81]:
emb.get_dimensionality()


235

In [82]:
from mirage.index.vector_index.ram_vector_index import L2RAMVectorIndex

vector_index = L2RAMVectorIndex(dimensionality = emb.get_dimensionality())
emb.convert_chunks_to_vector_index(chunks, vector_index, visualize=True)


Converting ChunkStorage to VectorIndex


100%|██████████| 38/38 [00:00<00:00, 1899.57it/s]


In [95]:
answer = vector_index.query(
    emb.embed(
        "какая еда полезна?"
    ), top_k=5
)
[chunks[answer[i].chunk_storage_key] for i in range(5)]

['у дома. Велосипеды популярны в городе. Велосипедная прогулка полезна для',
 'для поездок. Трамвайный маршрут проходит через центр. Автобус остановился на',
 'Молоко полезно для здоровья. Молочный коктейль любят дети. Сыр лежит',
 'сидения. Стульный дизайн может быть разным. Кровать стоит в спальне.',
 'перевозят грузы и людей. Поездный вокзал всегда busy. Велосипед стоит']

'торцевым освещением, состоящие из двух или более\nприборов в общем корпусе, привязные, поясные и плечевые ремни, швартовочное оборудование для\nгрузов и багажа, включая контейнеры, ящики, поддоны и т.п., применяемые в кабинах для пассажиров и\nэкипажа, должны иметь скорость горения не выше 60 мм/мин при испытании их на горючесть в\nгоризонтальном положении в соответствии с относящимися к ним пунктами Приложения F к настоящим\nНормам или другими эквивалентными методиками.\n(v) Кроме изоляции электрической проводки и небольших деталей (таких, как кнопки, рукоятки,\nштурвалы, замки, зажимы, люверсы, прокладки, шкивы и небольшие детали электрооборудования),\nкоторые мало способствуют распространению пламени, материалы изделий, не оговоренных вподпунктах\n(d)(3)(i), (ii) или (iv) настоящего параграфа, должны иметь скорость горения'